In [1]:
# Dependencies
import pandas as pd
import pymongo

from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

#import requests
#from selenium import webdriver

In [2]:
# Setting up splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
INFO:WDM:====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
INFO:WDM:Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
INFO:WDM:Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/102.0.5005.61/chromedriver_win32.zip
INFO:WDM:About to download new driver from https://chromedriver.storage.googleapis.com/102.0.5005.61/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\Ray\.wdm\drivers\chromedriver\win32\102.0.5005.61]
INFO:WDM:Driver has been saved in cache [C:\Users\Ray\.wdm\drivers\chromedriver\win32\102.0.5005.61]


In [3]:
# URL to scrape
url_news = 'https://redplanetscience.com/'
url_imgs = 'https://spaceimages-mars.com'

In [4]:
# Working with redplanetscience
browser.visit(url_news)

In [6]:
# HTML object
html = browser.html

# Using Beautiful Soup
soup_news = bs(html,'html.parser')

# Identifying elements with needed information
article = soup_news.find('div', class_='list_text')

In [11]:
# Saving information to variables
news_title = article.find('div', class_='content_title').text
news_text = article.find('div', class_='article_teaser_body').text

In [12]:
# Exiting browser
browser.quit()

In [ ]:
# Working with spaceimages
browser.visit(url_imgs)

In [16]:
#results = soup_news.find('section', class_='image_and_description_container')

#imgs = requests.get(url_imgs)
#soup_news = bs(news.text, 'lxml')
#soup_imgs = bs(imgs.text, 'html.parser')

result_news = soup_news.find('div', id_='news', class_='container')

print(result_news)

None


In [22]:
#driver = webdriver.Firefox()

#driver.get('https://redplanetscience.com/')

# PATH errors: selenium.common.exceptions.WebDriverException: Message: 'geckodriver' executable needs to be in PATH.

In [ ]:
#('div', class_='content_title').text
#('div', class_='article_teaser_body').text

In [32]:
#t = results.find_all('div', class_='col-md-12')
#print(t)


[<div class="container" id="news">
</div>]
